In [3]:
import time
import math
import numpy as np
import sympy as sp
from sympy.utilities.lambdify import lambdify
from scipy.optimize import minimize

## Forward Kinematics & IK & Jacobian (Joint space -- Foot space)

In [5]:
def T(theta, x, y):
    return sp.Matrix([[sp.cos(theta), -sp.sin(theta), x], 
                      [sp.sin(theta), sp.cos(theta), y],
                      [0, 0, 1]])

(thetaL_sym, thetaR_sym) = sp.symbols("""thetaL_sym thetaR_sym """, real = True)

l1 = 0.09
l2 = 0.16
w  = 0.07

def RobotKinematics(l1 = l1, l2 = l2, w = w):

    x_r = -w/2 + l1*sp.cos(thetaR_sym)
    x_l = w/2 + l1*sp.cos(thetaL_sym)
    y_r = l1*sp.sin(thetaR_sym)
    y_l = l1*sp.sin(thetaL_sym)
    theta3_sym = sp.atan2(y_l - y_r, x_l - x_r)
    L = sp.sqrt((x_l - x_r)**2 + (y_l - y_r)**2)

    FK = T(thetaL_sym, w/2, 0)@T((sp.pi - thetaL_sym + theta3_sym), l1, 0)@sp.Matrix([L/2, -sp.sqrt(l2**2 - (L/2)**2), 1])
    FK = FK[:2,:]
    J = FK.jacobian([thetaR_sym, thetaL_sym]).evalf()

    return FK, J
    
def InvereseKinematics(x, y, l1 = l1, l2 = l2, w = w):

    def leg_wide(var):
        return np.linalg.norm([var[0], var[1] - np.pi]) 

    def x_constraint_equation(var):
    # should be equal to zero when the 
        return l1**2 - l2**2 + (x - w/2)**2 + y**2 - 2*l1*(y*np.sin(var[0]) + (x - w/2)*np.cos(var[0]))

    def y_constraint_equation(var):
        return l1**2 - l2**2 + (x + w/2)**2 + y**2 - 2*l1*(y*np.sin(var[1]) + (x + w/2)*np.cos(var[1]))

    res = minimize(leg_wide, (0.1, 9*np.pi/10), method="SLSQP", constraints= ({"type": "eq", "fun": x_constraint_equation}, 
                                                                              {"type": "eq", "fun": y_constraint_equation}))
    return (res.x[0], res.x[1])

FK = RobotKinematics()[0]
FK_fast = lambdify((thetaR_sym, thetaL_sym), FK) # (x,y) = FK_fast(thetaR,thetaL)
J = RobotKinematics()[1]
J_fast = lambdify((thetaR_sym, thetaL_sym), J) # J_fast(thetaR,thetaL)

## Currents For Anti-Gravity

In [ ]:
'''
These are the average currents of 3 test cycles, each cycle has 231 data points.
Method to measured the current: Position control mode with pos_gain = 60, vel_integraotr_gain = 0.00025
'''
avg_current_m0 = np.array([0.24385765194892883, 0.3094293673833211, 0.7187509934107462, 0.6929809252421061, 1.3647583325703938, 1.7217425505320232, 1.300304889678955, 1.648040533065796, 1.9535056352615356, 1.4974501132965088, 2.31532613436381, 1.9920955499013264, 2.2254158655802407, 2.3666520913441977, 1.847597360610962, 1.6390361388524373, 1.6461957693099976, 2.1099076668421426, 2.1325478553771973, 1.8163514137268066, 1.8888504107793171, 2.9206464290618896, 2.6543094317118325, 2.0987207889556885, 2.092839320500692, 1.999744137128194, 2.2282211780548096, 1.6792758305867512, 1.10627414782842, 1.4703027407328289, 2.1606341203053794, 1.5280054410298665, 1.7889078855514526, 0.6789019306500753, 1.1961313486099243, 0.48277286688486737, 0.612623006105423, 1.1495956977208455, 1.1725813150405884, -0.2772788455088933, 0.332332968711853, 0.6521611015001932, 0.7043842375278473, 0.6808854738871256, 1.1591075261433919, 1.054255445798238, 0.743113378683726, 0.9329129457473755, 1.0790797074635823, 1.0422943433125813, 2.032093127568563, 1.8056693871815999, 2.0906312465667725, 0.849586566289266, 1.029306689898173, 2.0222384929656982, 1.9436176617940266, 2.126174290974935, 2.641770919164022, 2.1976657708485923, 2.1059587001800537, 2.2957820892333984, 2.3773279984792075, 2.4455095132191977, 1.975038210550944, 2.00803812344869, 1.8102043867111206, 1.9721685250600178, 2.4267675081888833, 2.6304352283477783, 1.8454880317052205, 1.744606852531433, 1.2482138872146606, 1.4302936792373657, 2.188453753789266, 1.5512415568033855, 1.631245772043864, 0.6038171450297037, 0.13339702288309732, 0.4121633768081665, 1.143872857093811, 0.14988629023234049, -0.7167718211809794, 0.22382322947184244, -0.4171148935953776, 0.511415441830953, 1.2329217195510864, 1.3153606255849202, 1.2904865741729736, 0.9308393001556396, 1.7994898955027263, 2.0531906286875405, 2.3358784914016724, 2.161727865537008, 1.9283878405888875, 2.015161315600077, 2.5283345381418862, 2.1249244610468545, 2.5240070819854736, 2.0664217869440713, 1.9704421758651733, 2.0232108434041343, 2.265923102696737, 2.258591572443644, 2.3936071395874023, 2.6251205603281655, 2.5077221393585205, 2.4912234942118325, 2.473044236501058, 2.0820539792378745, 2.2661094268163047, 1.745513677597046, 3.1524131298065186, 1.9257471561431885, 2.4136871496836343, 1.8905646800994873, 1.6534742911656697, 1.7814180850982666, 1.9101956288019817, 1.9164384206136067, 1.0571859280268352, 0.4606725573539734, 1.091152509053548, 1.476565678914388, 0.3367473979791005, -0.47332294782002765, 0.5010560154914856, 1.1517316897710164, 1.5442677736282349, 1.6470460096995037, 1.3049650986989338, 1.9339747428894043, 1.8914369742075603, 1.0414698521296184, 1.4385973612467449, 1.969805081685384, 2.447709321975708, 1.7240252097447712, 1.928009271621704, 1.8547149499257405, 2.0282653172810874, 2.594468593597412, 2.1630724668502808, 2.127838452657064, 3.202412764231364, 2.6751373608907065, 2.051024317741394, 2.498440663019816, 2.4904049237569175, 2.088454325993856, 1.5678453644116719, 1.406442642211914, 2.250063101450602, 1.866146167119344, 2.304741621017456, 1.309440056482951, 1.4436272780100505, 1.2946491241455078, 1.7087289492289226, 1.3948681354522705, 0.19499091307322183, 1.7129745086034138, 1.5390563408533733, 0.5865997423728307, 0.6500785102446874, 0.4878290295600891, 1.2354771693547566, 0.6837249596913656, 0.8018242716789246, 0.9551982680956522, 0.1827294925848643, 0.5636499921480814, 0.49909430742263794, 1.643161416053772, 1.1430108547210693, 1.1951600313186646, 1.9328319231669109, 2.168688098589579, 2.1448940436045327, 1.781290332476298, 2.3837035497029624, 2.772059122721354, 1.4827723900477092, 1.9108161528905232, 1.4763656457265217, 1.8926761547724407, 2.33011531829834, 2.10691237449646, 2.256103992462158, 1.145057241121928, 1.4253995815912883, 1.5443984667460124, 1.977695902188619, 2.0023202896118164, 1.9223989248275757, 1.4143034219741821, 2.206624666849772, 2.071223735809326, 1.874717076619466, 1.8417784770329793, 1.8987433910369873, 2.293064753214518, 1.4327460527420044, 1.5438967148462932, 1.7068617741266887, 1.2466908693313599, 0.09146864463885625, 0.14497573177019754, 0.9241856733957926, -0.12775465846061707, -0.10185965398947398, 1.191097617149353, 1.0708035031954448, 0.5770930250485738, 1.7961201270421345, 1.6440867980321248, 1.2170868714650471, 1.90204922358195, 1.5947521527608235, 1.5382156769434612, 2.3090309302012124, 1.1101342837015789, 1.9734142621358235, 2.0313257376352944, 0.5989607771237692, 1.7254005273183186, 1.169793168703715, 1.6195868253707886, 1.487960656483968, 1.6649297873179119, 1.2752639849980671])
avg_current_m1 = np.array([-2.2797098954518638, -2.053180376688639, -2.3361454010009766, -2.3334283034006753, -2.081712563832601, -1.3692905108133953, -1.9024568398793538, -1.7690024773279827, -1.3322250843048096, -1.994081695874532, -1.6818673610687256, -1.8202637434005737, -1.1361668109893799, -0.5988062421480814, -1.048218051592509, -0.6973060766855875, -0.8300726811091105, -0.7380097508430481, -0.11959755669037501, 0.14762013157208762, 0.3353431175152461, -0.6869473854700724, -0.5016278525193533, -0.7577384114265442, -1.2399080594380696, -1.340093731880188, -1.3555967807769775, -0.666299303372701, -1.4233221213022869, -2.193217118581136, -2.1028990745544434, -2.103356917699178, -1.6393461624781291, -1.859449823697408, -1.269327203432719, -2.348478396733602, -2.4404971599578857, -2.3028586705525718, -2.7627042134602866, -2.24586812655131, -2.335736115773519, -2.295964320500692, -2.1735856533050537, -2.0345710118611655, -2.249800523122152, -2.0066835085550943, -2.2386058966318765, -2.7825369834899902, -2.1118180751800537, -2.2224119504292807, -1.6119219462076824, -1.4509632587432861, -1.7773346900939941, -2.1761478583017984, -1.5867713292439778, -1.5122490723927815, -0.9182511766751608, -0.2192233751217524, -1.308614174524943, -1.0779201189676921, -0.6103412806987762, 0.266292467713356, 0.3024089535077413, -0.3552477955818176, -1.0995302597681682, -1.2309869925181072, -1.4033260742823284, -0.9891932805379232, -1.2713457743326824, -1.2932180563608806, -1.8089810212453206, -2.281296650568644, -1.770991365114848, -1.479876200358073, -2.0801594654719033, -2.035137176513672, -2.664177497227987, -2.2222375869750977, -2.280874570210775, -2.070718685785929, -2.1321224371592202, -2.0859897136688232, -2.294501622517904, -2.3432594935099282, -2.8702147006988525, -2.5244043668111167, -2.0505572160085044, -1.70535413424174, -2.203084707260132, -1.996114452679952, -2.0780444145202637, -1.9735807577768962, -2.3072122732798257, -1.6672333876291912, -1.9841952323913574, -1.4278603792190552, -1.5234063069025676, -1.5701942443847656, -1.460862159729004, -0.7478309075037638, -0.3853119711081187, -1.3322337865829468, -1.1077086130777996, -0.41606027881304425, 0.45204410950342816, -0.6481122374534607, -1.087979833285014, -1.4994410276412964, -0.9439000487327576, -1.322035789489746, -1.6403996149698894, -1.7279959917068481, -1.6262753009796143, -1.8283069531122844, -1.9439634482065837, -2.119074741999308, -2.2233635584513345, -2.0330498218536377, -1.8607685565948486, -2.2394651571909585, -1.783077319463094, -2.394257386525472, -2.329181512196859, -2.095521926879883, -2.630920966466268, -2.694347381591797, -2.2236881256103516, -2.0829123655954995, -1.7431803544362385, -1.8088794549306233, -2.2208834489186606, -2.0757440328598022, -2.420562744140625, -1.7030529181162517, -1.9283218781153362, -1.9219011068344116, -2.2662204106648765, -1.8367143472035725, -1.1591898202896118, -1.3650212685267131, -1.5507978200912476, -1.5597504774729412, -0.8181673884391785, -0.06766634806990623, -1.1718734502792358, -0.7329153219858805, -0.2809777061144511, -0.5715944568316141, -1.4501049915949504, -0.9978561004002889, -0.9296558300654093, -1.458973487218221, -1.8228953679402669, -1.123289982477824, -1.5559649467468262, -2.038268248240153, -2.7279580434163413, -1.8738028605779011, -1.8974505265553792, -1.5848116079966228, -2.53731902440389, -2.2780799070994058, -1.5509647925694783, -1.8995410203933716, -1.8916672468185425, -2.0144755442937217, -2.4499480724334717, -2.430097977320353, -2.221113443374634, -2.1024104754130044, -1.808496117591858, -2.144047657648722, -2.1213815609614053, -1.4347606499989827, -1.6695444583892822, -1.9843899408976238, -1.9912807941436768, -1.2884901960690816, -1.8188876310984294, -1.8037696679433186, -1.7592846155166626, -1.1740437348683674, -1.3681784868240356, -1.5993361473083496, -1.5554521083831787, -0.6082061529159546, -0.46297821402549744, -0.6529894868532816, -0.4714416066805522, -0.7206048369407654, -0.862498422463735, -0.3330726313094298, -1.0443281531333923, -1.5748084783554077, -1.7589613199234009, -1.392319917678833, -2.1246463457743325, -1.7696581681569417, -1.4065850575764973, -1.3755933443705242, -1.7766231298446655, -1.4913920561472576, -1.4969498713811238, -2.1290204524993896, -1.8802826404571533, -2.0364091396331787, -1.3890225092569988, -1.9366425673166912, -1.6821291049321492, -1.6241964896519978, -1.5163408120473225, -1.2886511882146199, -1.2609231074651082, -1.4418997764587402, -1.183966875076294, -0.9812319080034891, -0.81937309106191, -1.6255487998326619, -1.6474672158559163, -1.1632002194722493, -1.9732113281885784, -1.4739481608072917, -1.424432675043742, -1.5124091307322185, -0.44272298614184064, -0.7685344616572062, -0.9675639669100443, -0.4578015406926473, 0.0446436715622743, -0.8034603794415792, -0.43346485992272693])

avg_current_m0 = avg_current_m0.reshape((11,21))
avg_current_m1 = avg_current_m1.reshape((11,21))

'''
Flip the even number rows 
'''
for row_index in range(len(avg_current_m0)):
    if row_index % 2 != 0:
        avg_current_m0[row_index, :] = np.flip(avg_current_m0[row_index, :])
        avg_current_m1[row_index, :] = np.flip(avg_current_m1[row_index, :])
    else:
        pass
    
from scipy import interpolate
x = np.arange(0, 21, 1)
y = np.arange(0, 11, 1)

z  = avg_current_m0
zz = avg_current_m1

f_m0_current = interpolate.interp2d(x, y, z)
f_m1_current = interpolate.interp2d(x, y, zz)

## Trajectory Generator

In [6]:
UpperAMP = 0.05
DownAMP = 0.02
StanceHeight = 0.18
StepLength = 0.10

class move(object):
    '''
    Trajectory
    '''
    def LeftSwingTrajectory(t_clock, period, UpperAMP = UpperAMP, StanceHeight = StanceHeight, StepLength = StepLength):

        percent = t_clock / period

        if (percent <= 1): # Swing
            x = -(StepLength / 2) + percent * StepLength
            y = -UpperAMP * np.sin(np.pi * percent) + StanceHeight  

        else: # Stay and do nothing
            x = StepLength / 2
            y = StanceHeight

        return np.array([[x], [y]])

    def LeftStanceTrajectory(t_clock, period, DownAMP = DownAMP, StanceHeight = StanceHeight, StepLength = StepLength):

        percent = t_clock / period

        if (percent <= 1): # Stance
            x = (StepLength / 2) - (percent) * StepLength
            y =  DownAMP * np.sin(np.pi * percent) + StanceHeight

        else: # Stay and do nothing
            x = -StepLength / 2
            y = StanceHeight

        return np.array([[x], [y]])


    def RightSwingTrajectory(t_clock, period, UpperAMP = UpperAMP, StanceHeight = StanceHeight, StepLength = StepLength):

        percent = t_clock / period

        if (percent <= 1): # Swing
            x = (StepLength / 2)- percent * StepLength
            y = -UpperAMP * np.sin(np.pi * percent) + StanceHeight  

        else: # Stay and do nothing
            x = -StepLength / 2
            y = StanceHeight

        return np.array([[x], [y]])

    def RightStanceTrajectory(t_clock, period, DownAMP = DownAMP, StanceHeight = StanceHeight, StepLength = StepLength):

        percent = t_clock / period

        if (percent <= 1): # Stance
            x = -(StepLength / 2) + (percent) * StepLength
            y =  DownAMP * np.sin(np.pi * percent) + StanceHeight

        else: # Stay and do nothing
            x = StepLength / 2
            y = StanceHeight

        return np.array([[x], [y]])
    '''
    Virtual compliance
    k  = np.array([[kx,0],[0,ky]])
    c  = np.array([[cx,0],[0,cy]])
    '''
    def anisotropic_antigravity_rightswing(kval, theta_R, theta_L, vel_R, vel_L, t_swing_start, period, sr):

        t_clock = time.time() - t_swing_start
        disp    = FK_fast(theta_R, theta_L) - move.RightSwingTrajectory(t_clock, period)
        Kt = 0.0285
        c  = np.array([[0.5,0],[0,0.5]])

        if disp[1] < -0.001: #deltaY value
            k = np.array([[kval ,0],[0,kval*sr]]) # sr = stiffness ratio    
        else:
            k = np.array([[kval,0],[0,kval]])

        vel     = J_fast(theta_R, theta_L) @ np.array([[vel_R], [vel_L]])
        current = -J_fast(theta_R, theta_L).T @ ( k @ disp + c @ vel )*(1/Kt) 

        return current

    def anisotropic_antigravity_rightstance(kval, theta_R, theta_L, vel_R, vel_L, t_stance_start, period):

        t_clock = time.time() - t_stance_start
        disp    = FK_fast(theta_R, theta_L) - move.RightStanceTrajectory(t_clock, period)
        Kt = 0.0285
        c  = np.array([[0.5,0],[0,0.5]])
        k = np.array([[1.2*kval,0],[0,1.2*kval]])

        vel     = J_fast(theta_R, theta_L) @ np.array([[vel_R], [vel_L]])
        current = -J_fast(theta_R, theta_L).T @ ( k @ disp + c @ vel )*(1/Kt) 

        return current


    def anisotropic_antigravity_leftswing(kval, theta_R, theta_L, vel_R, vel_L, t_swing_start, period, sr):

        t_clock = time.time() - t_swing_start
        disp    = FK_fast(theta_R, theta_L) - move.LeftSwingTrajectory(t_clock, period)
        Kt = 0.0285
        c  = np.array([[0.5,0],[0,0.5]])

        if disp[1] < -0.001: #deltaY value
            k = np.array([[kval ,0],[0,kval*sr]]) # sr = stiffness ratio 
        else:
            k = np.array([[kval,0],[0,kval]])

        vel     = J_fast(theta_R, theta_L) @ np.array([[vel_R], [vel_L]])
        current = -J_fast(theta_R, theta_L).T @ ( k @ disp + c @ vel )*(1/Kt) 

        return current

    def anisotropic_antigravity_leftstance(kval, theta_R, theta_L, vel_R, vel_L, t_stance_start, period):

        t_clock = time.time() - t_stance_start
        disp    = FK_fast(theta_R, theta_L) - move.LeftStanceTrajectory(t_clock, period)
        Kt = 0.0285
        c  = np.array([[0.5,0],[0,0.5]])  
        k = np.array([[1.2*kval,0],[0,1.2*kval]])

        vel     = J_fast(theta_R, theta_L) @ np.array([[vel_R], [vel_L]])
        current = -J_fast(theta_R, theta_L).T @ ( k @ disp + c @ vel )*(1/Kt) 

        return current